# <font color=darkcyan> Multivariate linear regression - Lasso </font>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors

It is assumed that for all $1\leqslant i \leqslant n$, 

$$
Y_i = X^\top_i \beta_{\star} + \varepsilon_i\,,
$$

where the $(\varepsilon_i)_{1\leqslant i\leqslant n}$ are i.i.d. random variables in $\mathbb{R}$, $X_i\in\mathbb{R}^d$ and $\beta_{\star}$ is an unknown vector in $\mathbb{R}^d$. Let $Y\in\mathbb{R}^n$ (resp. $\varepsilon\in\mathbb{R}^n$)  be the random vector such that  for all $1\leqslant i \leqslant n$, the $i$-th component of $Y$ (resp. $\varepsilon$) is $Y_i$ (resp. $\varepsilon_i$) and $X\in\mathbb{R}^{n\times d}$ the matrix with line $i$ equal to $X^\top_i$. The model is then written

$$
Y = X \beta_{\star} + \varepsilon\,.
$$

In this section, it is assumed that $\mathbb{E}[\varepsilon] = 0$ and $\mathbb{E}[\varepsilon \varepsilon^\top] = \sigma_{\star}^2 I_n$. The Lasso estimate of $\beta_{\star}$ is defined as a solution to

$$
\widehat \beta_n\in  \mathrm{argmin}_{\beta\in\mathbb{R}^d}\,\left( n^{-1}\|Y - X\beta\|_2^2 + \lambda \|\beta\|_1\right)\,.
$$

where $\lambda>0$.

<font color=darkred> Explain the coordinate-wise optimization procedure </font>

A coordinate descent can be applied to solve the LASSO optimization problem. In this case, solving the Lasso optimization problem amounts to producing iterative estimators, where at each iteration, a coordinate is selected to be updated. Then, the objective function  is optimized explicitly  with respect to the selected coordinate. For all $\beta \in \mathbb{R}^d$,  
$$
\nabla_\beta \|Y - X\beta\|_2^2 = - 2 X^\top (Y-X\beta)\,.
$$
Then, for all $1\leqslant j \leqslant d$, $(\nabla_\beta \|Y - X\beta\|_2^2)_j = -2 {\bf X}^\top_j (Y-X\beta)$, where ${\bf X}_j$ is the $j$-th column of the matrix $X$. 
Define, for all $1\leqslant j \leqslant d$,
$$
\upsilon_{j}={\bf X}^\top_{j}\left(Y-\sum_{\substack{i=1\\ i\neq j}}^d\beta_{i}{\bf X}_{i}\right)\,.
$$
Assuming that the columns of $X$ are normalized, i.e. for all $1\leqslant k \leqslant d$, ${\bf X}^\top_{k}{\bf X}_{k}=1$, yields
$$
(\nabla_\beta \|Y - X\beta\|_2^2)_j = -2( \upsilon_j - \beta_j)\,.
$$
Consequently, for all $\beta_j \neq 0$, 
$$
(\nabla_\beta ( n^{-1}\|Y - X\beta\|_2^2 +  \lambda\|\beta\|_1))_j= \frac{2}{n}( \beta_j - \upsilon_j + \lambda n\textrm{sign}(\beta_j)/2)\,.
$$
For all $1\leqslant j\leqslant d$,  $\beta_j \mapsto  n^{-1}\|Y - X\beta\|_2^2 + \lambda\|\beta\|_1$ is convex and grows to infinity when $|\beta_j|\to \infty$ and admits thus a minimum at some $\beta_j^{\star}\in\mathbb{R}$. 

- If $\beta_j^{\star} \neq 0$, then
$$
\beta_j^{\star} = \upsilon_j\left( 1 - \frac{\lambda n~\textrm{sign}(\beta_j^{\star})}{2 \upsilon_j}\right)\,,
$$
which yields, as  $\textrm{sign}(\beta_j^{\star}) = \textrm{sign}(\upsilon_j)$,
$$
\beta_j^{\star} = \upsilon_j\left(1 - \frac{\lambda n }{2 |\upsilon_j|}\right)
$$
and
$$
1 - \frac{\lambda n }{2 |\upsilon_j|} \geqslant 0\,.
$$
- If $1 - \lambda n/(2 |\upsilon_j|)<0$, there is no solution to $(\nabla_\beta ( n^{-1}\|Y - X\beta\|_2^2 +  \lambda\|\beta\|_1))_j=0$ for $\beta_j \neq 0$.  Since $\beta_j \mapsto  n^{-1}\|Y - X\beta\|_2^2 + \lambda\|\beta\|_1$ admits a minimum, $\beta_j^{\star}=0$. 
Therefore,
$$
\beta_j^{\star} = \upsilon_j\left( 1 - \frac{\lambda n}{2 |\upsilon_j|}\right)_+ = \mathrm{max}\left(0;\upsilon_j\left( 1 - \frac{\lambda n}{2 |\upsilon_j|}\right)\right)\,.
$$

As a conlusion:
- if $|\upsilon_j|\geq \lambda n / 2$, $\beta_j^{\star} = \upsilon_j( 1 - \frac{\lambda n}{2 |\upsilon_j|})$ and if $|\upsilon_j|< \lambda n / 2$, $\beta_j^{\star} = 0$ ;
- if $\upsilon_j\leq -\lambda n / 2$, $\beta_j^{\star} = \upsilon_j + \frac{\lambda n}{2}$,  if $\upsilon_j\geq \lambda n / 2$, $\beta_j^{\star} = \upsilon_j - \frac{\lambda n}{2}$, and if $|\upsilon_j|< \lambda n / 2$, $\beta_j^{\star} = 0$

#### Import data

In [ ]:
import pandas as pd

Data frames can be imported using pandas. This provides two-dimensional and heterogeneous tabular data.
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html

<font color=darkred>
Import data in the file BRinf using ``read_csv``, display the first rows with ``head`` and the shape of the dataframe using ``shape``.
</font>

In [ ]:
# In this section, multivariate linear regression is used to predic the Brazilian inflation based on
# many observed variables, see https://github.com/gabrielrvsc/HDeconometrics/
df = pd.read_csv('BRinf.txt')
df.head()

In [ ]:
# number of observations, number of variables
df.shape

<font color=darkred>
Use the ``StandardScaler`` of sklearn to preprocess the input variables.
</font>

``StandardScaler`` standardizes the input variables by removing the mean and scaling to unit variance.
We will not analyze closely standardization in this course. However, it is often very useful (even mandatory in some cases) for the stability of learning procedures.
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(df.iloc[:,2:])

In [ ]:
# first coordinate is the number of samples
# second coordinate is the number of input features (+ 1 for the observations)
np.shape(X)

<font color=darkred>
Build two datasets. 
    ``X_train`` and ``Y_train`` contain the first 140 input data and observations. ``X_test`` and ``Y_test`` contain the remaining input data and observations. We train a linear regression model using ``X_train`` and ``Y_train`` and we assess the performance of the model using ``X_test`` and ``Y_test``. 
</font>

https://pandas.pydata.org/docs/reference/frame.html

In [ ]:
nb_data_train = 140
# inflation observations
Y_train = df.iloc[:nb_data_train,1] 
Y_test  = df.iloc[nb_data_train:,1] 
Y_test.head()

In [ ]:
# other variables
X_train = X[:nb_data_train,:] 
X_test  = X[nb_data_train:,:] 
np.shape(X_train)

#### Lasso Regression from scractch

<font color=darkred>
Write a ``threshold_function`` function with arguments a real number ``z`` and a positive number ``$\alpha$`` which returns 
    
    - $z+\alpha$ if $z<-\alpha$,
    
    - $z-\alpha$ if $z>\alpha$,
    
    - 0 otherwise.
</font>

In [ ]:
def threshold_func(z,alpha):
    if z < - alpha:
        return (z + alpha)
    elif z >  alpha:
        return (z - alpha)
    else: 
        return 0

<font color=darkred>
    Write a ``coordinate_descent_lasso`` function with arguments an initial estimate ``$\beta$``, the data ``X`` and ``y``, a penalty parameter ``$\alpha$`` and a number of iterations ``n_iter``. The function returns the parameter estimate after n_iter iterations of the coordinate-wise optimization procedure.
    </font>

To make things simpler, you can write the function with $\alpha = \lambda n /2$ in the mathematical derivation above.

In [ ]:
def coordinate_descent_lasso(beta,X,y,alpha, n_iter):
    n,d = X.shape
    X = X / (np.linalg.norm(X,axis = 0)) 
    for i in range(num_iter): 
        for j in range(d):
            X_j = X[:,j].reshape(-1,1)
            y_pred = X @ beta
            r_j = X_j.T @ (y - y_pred  + beta[j]*X_j)
            beta[j] =  threshold_func(r_j, alpha)         
    return beta

<font color=darkred>
    Run the algorithm with several values of $\alpha$ using X_train and Y_train and display the number of zero coeeficients of the parameter estimate and the MSE obtained on the test set.
    </font>

#### Lasso Regression with Sklearn

In [ ]:
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV

<font color=darkred>
Create a np array with several values of the penalty parameter (called $\alpha$ in Python)
</font>

In [ ]:
n_alphas = 100
alphas = np.logspace(-10, -1, n_alphas)

<font color=darkred>
Use the ``fit`` function of sklearn to fit a Lasso model with for each value of $\alpha$. 
    
Store the estimated parameter, the number of zeros in the estimated parameter and the MSE on the test set after each training.
</font>

In [ ]:
lasso = Lasso(max_iter=10000, normalize=True)
coefs = []
nb_zeros = []
rmse = []
for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, Y_train)
    coefs.append(lasso.coef_)
    nb_zeros.append(np.count_nonzero(lasso.coef_==0))
    y_hat = lasso.predict(X_test)
    rmse.append(mean_squared_error(y_hat,Y_test.values))


<font color=darkred>
Display the estimated parameters as a function of the penalty parameter.
</font>

In [ ]:
plt.figure(1,figsize=(8,8))
ax = plt.gca()
ax.plot(alphas, coefs)
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel("lambda")
plt.ylabel(r'$\widehat\theta^{\mathrm{lasso}}_{n,\lambda}$')
plt.title("Lasso coefficients as a function of the regularization")
plt.axis("tight")
plt.show()

<font color=darkred>
Display the number of zero coefficients of the estimated parameter as a function of the penalty parameter.
</font>

In [ ]:
plt.figure(1,figsize=(8,8))
ax = plt.gca()
ax.plot(alphas, nb_zeros)
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel("lambda")
plt.ylabel('Number of zero coefficients')
plt.title("Number of null coefficients as a function of the regularization")
plt.axis("tight")
plt.show()

<font color=darkred>
Display the MSE on the test set as a function of the penalty parameter.
</font>

In [ ]:
plt.figure(1,figsize=(8,8))
ax = plt.gca()
ax.plot(alphas, rmse)
ax.set_xscale("log")
ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
plt.xlabel("lambda")
plt.ylabel('Mean squared error')
plt.title("Mean squared error as a function of the regularization")
plt.axis("tight")
plt.show()